# USA Vin Profitability Load Job Procedure

### This notebook contains the process steps required to update the Vin Profitability Biq Query Tables
### Created By:  Richard Honey  on  14th April 2017 

In [126]:
from google.datalab import Context
import google.datalab.bigquery as bq
import google.da talab.storage as storage
import pandas as pd
from StringIO import StringIO
import googleapiclient.discovery
#import gcp.bigquery as bq2

In [ ]:
def create_transfer_client():
    return googleapiclient.discovery.build('storagetransfer', 'v1')

## General ledger append latest data from eSMART Fact Table

### Define Schema for eSMART General Ledger Fact Data for Wholesale

In [127]:
# Step 1 - Go int SAP BW (RSA1) - Find DSO J_FI_D26 - Finance Line Items (eSmart)
# Step 2 - Right Click on DSO and select display data
# Step 3 - On Characteristcis Selection - Goto 'Read from File' and collect the structure from 
#          Z:\Analytics\Wholesale\SAP_GL_eSMART_J_FI_D26_Extract_Structure.txt
# Step 4 - Select the following GL Accounts 
#             700050102  Variable Mktg Retail   Accrual Retail Accrual
#             900011600  Vehicles Sales Freight
#             901020100  Other Revenue Adjustments In Transit Floor Plan
#             901030100  Loss On Resale Company Vehicles
#             901050100  Variable Dealer Margin   Dealer Standards
#             901070200  Duties  Taxes on Revenue 1
#             1100030000 Duty  Import Taxes
#             1100070500 Oth Cenl Cost   Other Sales Dealers
#             1101010107 WAR Extended Cust
#             1101010301 WAR Recoveries Gen
#             1101010901 WAR Roadside  Gen
#             1101011300 WAR Ext Purchase CPO
#             1101030000 WAR Serv Plan Costs
#             1102020106 FRT Ob Sea USA
#             1102040100 FRT Prep PDI  OVC

#            
# Step 5 - Select company code to US02
# Step 6 - On specifications for return type - select "Store in File Workstation"
#          File location is Z:\Analytics\VIN Profitability\GL Costs Files\USA\GL_eSMART_US_MTH_YEAR.CSV
# Step 8 - Copy file to Google Storage bucket /sap_data/Wholesales/eSMART
#
# After completing above steps you can load the US General Ledger data to Big Query
finance_schema = bq.Schema([
{'name':'AU_VHVIN','type':'STRING'}, # VIN
{'name':'CURRENCY','type':'STRING'}, #Currency
{'name':'DOC_CURRCY','type':'STRING'}, # doc_currency
{'name':'LOC_CURRCY','type':'STRING'},# local_currency
{'name':'UNIT','type':'STRING'},# Unit
{'name':'ZGACDOCTP','type':'STRING'},# Document Type
{'name':'ZGASSNNUM','type':'STRING'}, #Assignment Number
{'name':'ZGGLACCNT','type':'STRING'}, # GL Account
{'name':'FISCPER','type':'STRING'}, # Fiscal Period
{'name':'ZGCOM_COD','type':'STRING'}, # Company Code
{'name':'AMOUNT','type':'STRING'}, # Amount
{'name':'DEB_CRE_DC','type':'STRING'}, # Debit/Credit Amount Document Currency
{'name':'DEB_CRE_LC','type':'STRING'}, # Debit/Credit Amount Local Currency
{'name':'DSC_AMT_DC','type':'STRING'}, # Discount Amount Document Currency
{'name':'DSC_AMT_lC','type':'STRING'}, # Discount Amount Local Currency
{'name':'SR_TAXESLC','type':'STRING'}, # Tax local Amount
{'name':'TAX_AMOUNT','type':'STRING'}, # Tax Amount
{'name':'ZGBALTPR','type':'STRING'}, # Base List Price
{'name':'ZGCOUNTER','type':'STRING'}, # Document Counter
{'name':'ZGVESTQTY','type':'STRING'} # Vehicle Stock Quantity
])

### Define Finance eSMART table to update

In [128]:
# Define the table to append the latest GL data to
Load_finance_table = bq.Table('US_Analytics.FACT_eSMART_GL_Costs_Raw_File')

### Load the eSMART GL data to biq query table 

In [129]:
Load_finance_table.load('gs://sap_data/Wholesales/eSMART/GL_eSMART_US_JAN_2016.CSV', mode='append',
                  source_format = 'csv',
                 csv_options=bq.CSVOptions(delimiter=u';',skip_leading_rows=0))

Job jlr-dl-dev/job_j1NgEeEG7328VBtVQyuZ-fIFgHs completed

### Transpose the eSMART GL monthly finance data into FACT_eSMART_GL_Costs_Transformed

In [130]:
%%bq query -n finance_transpose_data 
SELECT * FROM `US_Analytics.HLP_Finance_eSMART_Transpose_Standard`

### Run the query and materialize new transposed GL finance table
View used to transpose is HLP_Finance_eSMART_Transpose_Standard

In [140]:
%bq execute -q finance_transpose_data -t , --table US_Analytics.FACT_eSMART_GL_Costs_Transformed --mode='overwrite'

VIN,BBSS_Mark_Cost_Local,Delivery_Revenue_Local,Floor_Plan_Int_Local,Loss_On_Resale_Local,Var_Dealer_Margin_Local,Duties_Taxes,Duty_And_Imp_Taxes_Local,Port_Other_Prep_Local,Warr_Rav_Repairs_Local,Service_Loaner_Local,Warr_Roadside_Local,CPO_Warr_Local,Freight_OutB_Local,Dealer_PDI_Local,Base_Option_Ind,Wholesale_price_type
SALWR2VF2FA617842,750.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,B,M
SALWR2VF4FA523476,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,97.86,0.0,B,M
SALWG2VFXFA531099,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.37,0.0,0.0,0.0,0.0,0.0,0.0,B,M
SAJWJ0FF3F8U75303,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.52,0.0,0.0,0.0,0.0,0.0,0.0,B,M
SAJWJ0FF1F8U73517,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,359.0,0.0,B,M
SAJWA0F72F8U66701,2500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,B,M
SAJWJ0FF6F8U60827,2000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,B,M
SAJWA0HP3FMU67117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.08,0.0,0.0,0.0,0.0,576.0,0.0,B,M
SALWR2VF8FA619014,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,B,M
SALLSAAD4AA240184,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,389.18,0.0,0.0,B,M


## Chase Load -  add the latest quarter of USA chase data to the Big Query Raw Data Table

### Backup existing Big Query USA Chase Table

In [183]:
# Delete Backup so we can replace with new one
%bq tables delete -n 'US_Analytics.FACT_eSMART_Chase_Raw_File_Backup'

In [184]:
# Set the job name as current date
#!/usr/bin/python
 
from datetime import datetime
 
i = datetime.now()
 
ts =  i.strftime('%Y%m%d%H%M%S')

In [185]:
# Copy US_Analytics.FACT_eSMART_Chase_Raw_File to US_Analytics.FACT_eSMART_Chase_Raw_File_Backup for safety
from google.cloud import bigquery

client = bigquery.Client()
dataset = client.dataset('US_Analytics')
source_table = dataset.table('FACT_eSMART_Chase_Raw_File')
destination_table = dataset.table('FACT_eSMART_Chase_Raw_File_Backup')

import time

job = client.copy_table(ts, destination_table, source_table)

job.begin()

retry_count = 100
while retry_count > 0 and job.state != 'DONE':
  retry_count = 1
  time.sleep(10)
  job.reload()  # API call
print(job.state)
job.ended

DONE


datetime.datetime(2017, 4, 24, 9, 58, 16, 76000, tzinfo=<UTC>)

### Define the  chase load schema 

In [197]:
# Step 1 - Select the latest Chase File CHASE_USA_APR_MTH_YYYY (CHASE_USA_SEPT_2017) located in Storage Bucket /sap_data/Vin_Profitability/Loan System/USA   
Chase_schema = bq.Schema([
{'name':'VIN','type':'STRING'},#VIN
{'name':'Booking_Date','type':'STRING'},#Booking Date
{'name':'Subvention','type':'FLOAT'},#Subvention Amount    
{'name':'Subvention_Type','type':'STRING'},#Subvention Type
])

### Define the  USA Chase table to load data

In [198]:
# Define the table to recreate from appended Chase File
Load_Chase = bq.Table('US_Analytics.FACT_eSMART_Chase_Raw_File')

### Append the latest Chase CSV to the Biq Query Table
File Format: CHASE_USA_FROM_TO_YEAR.csv  (Example CHASE_USA_APR_SEPT_2017.csv)

In [200]:
Load_Chase.load('gs://sap_data/Vin_Profitability/CHASE_USA_APR_SEPT_2017.csv', mode='append',
                  source_format = 'csv',csv_options=bq.CSVOptions(delimiter=u',',skip_leading_rows=1))

Job jlr-dl-dev/job_E7DWD8p9wAC7gtVfUsneZRXNis0 completed

### Transpose the Chase Raw Data 

In [201]:
%%bq query -n chase_transpose_data 
SELECT * FROM `US_Analytics.HLP_Chase_eSMART_Transpose_Standard`

### Run the query and materialize new transposed US Chase table
View used to transpose is HLP_Chase_eSMART_Transpose_Standard

In [202]:
%bq execute -q chase_transpose_data -t , --table US_Analytics.FACT_eSMART_Chase_Transformed --mode='overwrite'

VIN,Chase_APR_Rate,Chase_Lease_Rate,Chase_RV,Chase_Pull_Ahead
SALGS2KF0GA305629,0.0,3603.08,0.0,0.0
SAJAR4BG6HA965089,0.0,2551.43,0.0,0.0
SALCR2BG9GH625400,0.0,3716.24,0.0,0.0
SALVP2BG0GH081996,0.0,4404.76,0.0,0.0
SALVP2BG9GH117877,0.0,5162.95,0.0,0.0
SALGS2PF0GA314792,0.0,7298.4,0.0,0.0
SALCR2BG8HH646823,0.0,1095.99,0.0,0.0
SALVD2BG5HH169159,0.0,6346.88,0.0,0.0
SAJAD4BG9HA958250,0.0,4016.62,0.0,0.0
SAJWA6BC1F8K10944,0.0,11673.23,0.0,0.0


## Vehicle standard costs -  reload the latest base and options derivative costs to big query tables

### Base costs - define the vehicle base standard schema to reload data to

In [207]:
# Step 1 - Collect the latest standard base costs from the markets  
# Step 2 - Upload the file into Z:/Analytics/Vin Profitability/Vehicle Standard Costs/
# Step 3 - Upload the file in Google data storage bucket sap_data/Vin_Profitability/ 


Base_Cost_schema = bq.Schema([
{'name':'Derivative','type':'STRING'},#Derivative
{'name':'Date_From','type':'STRING'},#Date From
{'name':'Date_To','type':'STRING'},#Date To    
{'name':'Base_Material_Cost','type':'FLOAT'},#Base Costs
])

In [208]:
# Define the table to OVERWRITE
Load_Base_Costs = bq.Table('US_Analytics.FACT_eSMART_Base_Costs' ).create(schema = Base_Cost_schema, overwrite = True)

In [210]:
# Load the Base Vehicle Cost CSV file into big query FACT_eSMART_Base_Costs table 
Load_Base_Costs.load('gs://sap_data/Vin_Profitability/Vehicle_Costs/eSMART VEHICLE  Costs - US Base Costs.csv', mode='overwrite',
                  source_format = 'csv', csv_options=bq.CSVOptions(delimiter=u',',skip_leading_rows=1))

Job jlr-dl-dev/job_wqMeIul99aP0rW5T57l5cu9VLUQ completed

### Options costs - define the vehicle options standard schema to reload data to

In [211]:
# Step 1 - Collect the latest standard option costs from the markets  
# Step 2 - Upload the file into Z:/Analytics/Vin Profitability/Vehicle Standard Costs/
# Step 3 - Upload the file in Google data storage bucket sap_data/Vin_Profitability/  

Options_Cost_schema = bq.Schema([
{'name':'Derivative','type':'STRING'},#Derivative
{'name':'Date_From','type':'STRING'},#Date From
{'name':'Date_To','type':'STRING'},#Date To    
{'name':'Options_Material_Cost','type':'FLOAT'},#Options Costs
])

In [212]:
# Define the table to OVERWRITE
Load_Options_Costs = bq.Table('US_Analytics.FACT_eSMART_Options_Costs' ).create(schema = Options_Cost_schema, overwrite = True)

In [213]:
# Load the Base Vehicle Cost CSV file into big query FACT_eSMART_Base_Costs table 
Load_Options_Costs.load('gs://sap_data/Vin_Profitability/Vehicle_Costs/eSMART VEHICLE  Costs - US Options Costs.csv', mode='overwrite',
                  source_format = 'csv', csv_options=bq.CSVOptions(delimiter=u',',skip_leading_rows=1))

Job jlr-dl-dev/job_oEoAZFuWT2pdslVuyCafyrbNap8 completed

# Central freight and warranty costs -  reload the central standard costs

### Freight costs - define the freight central standard schema to reload data

In [219]:
# Step 1 - Collect the latest freight costs from the markets  
# Step 2 - Upload the file into Z:/Analytics/Vin Profitability/Freight Central Costs
# Step 3 - Upload the file in Google data storage bucket sap_data/Vin_Profitability/ 
Freight_Cost_schema = bq.Schema([
{'name':'Model','type':'STRING'},#Model
{'name':'Date_From','type':'STRING'},#Date From   
{'name':'Date_To','type':'STRING'},#Date To       
{'name':'Fiscal_Year','type':'STRING'},#Year
{'name':'Freight_Costs','type':'FLOAT'},#Freight Costs
])

In [220]:
# Define the table to OVERWRITE
Load_Freight_Costs = bq.Table('US_Analytics.FACT_eSMART_Freight_Central_Costs_Raw' ).create(schema = Freight_Cost_schema, overwrite = True)

In [221]:
# Load the Freight Central Cost CSV file into big query FACT_eSMART_Freight_Central_Costs table 
Load_Freight_Costs.load('gs://sap_data/Vin_Profitability/Freight_Central/eSMART Vehicle  Costs - Freight_Central_Costs.csv', mode='overwrite',
                  source_format = 'csv', csv_options=bq.CSVOptions(delimiter=u',',skip_leading_rows=1))

Job jlr-dl-dev/job_00vSP-MwhaGBUwFF_pNp6hujyKg completed

### Summarize the Freight Central data

In [222]:
%%bq query -n Freight_summarize_data 
SELECT * FROM `US_Analytics.HLP_Freight_Central_eSMART_Standard`

In [223]:
%bq execute -q Freight_summarize_data -t , --table US_Analytics.FACT_eSMART_Freight_Central_Costs --mode='overwrite'

Model,Fiscal_Year,Freight_Costs
R3,2017,-2004.0
R8,2017,-1761.0
H2,2016,-1923.0
CV,2016,-2322.0
W5,2016,-2035.0
G2,2017,-1952.0
Y4,2017,-1879.0
V3,2016,-2322.0
X7,2016,-1726.0
CV,2017,-2792.0


### Warranty costs - define the central standard schema to reload data

In [226]:
# Step 1 - Collect the latest warranty costs from the markets  
# Step 2 - Upload the file into Z:/Analytics/Vin Profitability/Warranty Central Costs
# Step 3 - Upload the file in Google data storage bucket sap_data/Vin_Profitability/ 

Warranty_Cost_schema = bq.Schema([
{'name':'Model','type':'STRING'},#Model
{'name':'Date_From','type':'STRING'},#Date From   
{'name':'Date_To','type':'STRING'},#Date To       
{'name':'Fiscal_Year','type':'STRING'},#Year
{'name':'Warranty_Costs','type':'FLOAT'},#Warranty Costs
])

In [227]:
# Define the table to OVERWRITE
Load_Warranty_Costs = bq.Table('US_Analytics.FACT_eSMART_Warranty_Central_Costs_Raw' ).create(schema = Warranty_Cost_schema, overwrite = True)

In [229]:
# Load the Warranty Central Cost CSV file into big query FACT_eSMART_Warranty_Central_Costs table 
Load_Warranty_Costs.load('gs://sap_data/Vin_Profitability/Warranty_Central/eSMART Vehicle  Costs   - Warraty_Central_Costs.csv', mode='overwrite',
                  source_format = 'csv', csv_options=bq.CSVOptions(delimiter=u',',skip_leading_rows=1))

Job jlr-dl-dev/job_Br5aW_OH0yWWDNgVOoeBf5wAcDU completed

### Summarize the Warranty Central data

In [230]:
%%bq query -n Warranty_summarize_data 
SELECT * FROM `US_Analytics.HLP_Warranty_Central_eSMART_Standard`

In [231]:
%bq execute -q Warranty_summarize_data -t , --table US_Analytics.FACT_eSMART_Warranty_Central_Costs --mode='overwrite'

Model,Fiscal_Year,Warranty_Costs
Y4,2016,-3376.0
X2,2016,-1971.0
G2,2017,-1691.0
X1,2016,-1982.0
M1,2017,-1988.0
H2,2016,-1771.0
CJ,2016,-4205.0
T1,2017,-2430.0
X7,2017,-2285.0
N1,2016,-961.0


## Retail Sales  - Last retail date  and original dealer -> full reload from January 2015

 Smith, Brian <bsmith3@jaguarlandrover.com will provide data

### Retail Data  - define the retail raw data schema to reload data

In [232]:
# Step 1 - Collect the latest retail from the markets  
# Step 2 - Upload the file into Z:/Analytics/Vin Profitability/Retail Data
# Step 3 - Upload the file in Google data storage bucket sap_data/Vin_Profitability/ 


Retail_schema = bq.Schema([
{'name':'VIN','type':'STRING'},#Model
{'name':'Retail_Sales','type':'INTEGER'},#Date From   
{'name':'Date','type':'STRING'},#Date To       
{'name':'Dealer','type':'STRING'},#Year
])

In [236]:
# Define the table to OVERWRITE
Load_Retail_Sales = bq.Table('US_Analytics.FACT_eSMART_US_Retail_Sales_Raw' ).create(schema = Retail_schema, overwrite = True)

In [237]:
# Load the Retail Cost CSV file into big query FACT_eSMART_Retail_Sales_Raw
Load_Retail_Sales.load('gs://sap_data/Vin_Profitability/Retail_Sales/eSMART USA Retail Sales Data.csv', mode='overwrite',
                  source_format = 'csv', csv_options=bq.CSVOptions(delimiter=u',',skip_leading_rows=1))

Job jlr-dl-dev/job_1G8vUi57QX7OkTAqkD9wykDn2Ao completed

### Summarize the Retail Sales data

In [241]:
%%bq query -n Retail_summarize_data 
SELECT * FROM `US_Analytics.HLP_Retail_Sales_eSMART_Standard`

In [242]:
%bq execute -q Retail_summarize_data -t , --table US_Analytics.DIM_eSMART_US_Retail_Max --mode='overwrite'

VIN,Retail_Qty,Retail_Date,Partner
SAJBE4BV1GCY09625,1,01/03/2016,J00517
SAJWA6BU2G8K32588,1,01/03/2016,J00517
SAJWJ6DL4GMK30384,1,01/03/2016,J00517
SAJWA2G73G8V95462,1,01/03/2016,J05537
SAJWA2G77G8V90961,1,01/03/2016,J05577
SALVP2BG4GH089079,1,01/03/2016,R00121
SALGS2VF7GA273595,1,01/03/2016,R00175
SALGS2EF0GA277178,1,01/03/2016,R00178
SALCR2BG3FH542138,1,01/03/2016,R00197
SALAG2V60GA812906,1,01/03/2016,R00209
